# Classify satellite images into building footprints 
If everything worked so far, we can then move on to actually classify an image provided we have enough confidence in the trained model. In this section, we will now load an image that we truly want to classify for real-world applications.

This code runs only with images that is ~5.5GB of disk space.
Bigger images will crash the notebook.

*Version: 0.2*

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
# Import libraries
import os, glob
import gdal
import numpy as np
import random
import math

import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches

!pip install rasterio
import rasterio
from rasterio.windows import Window

import tensorflow as tf
import keras
from tensorflow.python.keras import backend as K

import seaborn as sea

!pip install ipython-autotime
%load_ext autotime

!pip install tqdm
from tqdm import trange

     |████████████████████████████████| 19.1MB 301kB/s 
time: 2.84 s (started: 2021-04-11 18:47:47 +00:00)


## Load pre-requisite functions and models for predictions

In [ ]:
# Call the metrics and model module

from Model import *
from Metrics import *

time: 3.22 s (started: 2021-04-11 18:52:00 +00:00)


In [ ]:
# Load the model that you want to use for prediction
from keras.models import load_model

model = load_model("/content/drive/MyDrive/Python Elective Project/Checkpoints/LAST_SAVED_MODEL_15_12_1e-05.hdf5", 
                   custom_objects={"tversky": tversky, "f1_m": f1_m, "accuracy": accuracy, "precision_m": precision_m, "recall_m": recall_m}, compile=True) 

time: 1.51 s (started: 2021-04-11 18:52:23 +00:00)


## Load the images to classify

In [ ]:
#Import the image to classify
path = "/content/drive/MyDrive/Python Elective Project/Prediction Phase/Images to classify"
img_dir = os.path.join(path, "A1.tif")

#Load the image to classify
image = gdal.Open(img_dir)
bands_test = [image.GetRasterBand(i+1).ReadAsArray() for i in trange(image.RasterCount)]
new_image = np.stack(bands_test, axis=2)       
del bands_test

# To store the meta data, open with Rasterio
src = rasterio.open(img_dir)

100%|██████████| 3/3 [01:05<00:00, 21.89s/it]


time: 1min 9s (started: 2021-04-03 17:57:28 +00:00)


In [ ]:
#CIEW
print(f"Shape of the satellite image{new_image.shape}")

Shape of the satellite image(25088, 30208, 3)
time: 1.4 ms (started: 2021-04-03 17:58:38 +00:00)


## Patch generation and classification

In [ ]:
# Patch the image into 512x512 to predict

def gridwise_sample(imgarray, patchsize):

    """Extract sample patches of size patchsize x patchsize from an image (imgarray) in a gridwise manner.
    """
    nrows, ncols, nbands = imgarray.shape
    patchsamples = np.zeros(shape=(0, patchsize, patchsize, nbands),
                            dtype=imgarray.dtype)
    for i in trange(int(nrows/patchsize)):
        for j in trange(int(ncols/patchsize)):
            tocat = imgarray[i*patchsize:(i+1)*patchsize,
                             j*patchsize:(j+1)*patchsize, :]
            tocat = np.expand_dims(tocat, axis=0)
            patchsamples = np.concatenate((patchsamples, tocat),
                                          axis=0)
    return patchsamples

time: 5.13 ms (started: 2021-04-03 15:22:22 +00:00)


In [ ]:
# GENERATE PATCH TILES OF THE IMAGE
PATCHSIZE = 512

# Sample each tile systematically in a gridwise manner
patch = gridwise_sample(new_image, PATCHSIZE)

100%|██████████| 49/49 [25:41<00:00, 31.46s/it]

time: 25min 41s (started: 2021-04-03 15:22:22 +00:00)


In [ ]:
# Select the model and predict on the test image
prediction = model.predict(patch) # Data is float
print("The predictions were tested on %i number patches." % (patch.shape[0]))

The predictions were tested on 2891 number patches.
time: 1min 11s (started: 2021-04-03 15:48:03 +00:00)


In [ ]:
# To turn the predicted data from a float data type into an integer data type 
"""
ONLY NECESSARY IF YOU NEED TO CONVERT THE RASTERS TO POLYGONS AS YOU NEED IT IN INTEGER DATA TYPE
"""
prediction[prediction>=0.5]=1
prediction[prediction<0.5]=0

time: 2.56 s (started: 2021-04-03 15:49:14 +00:00)


In [ ]:
# CIEW
print(f"Values in the predictions are: {np.unique(prediction)}")

Values in the predictions are: [0. 1.]
time: 28.8 s (started: 2021-04-03 15:52:40 +00:00)


## Save and Load as Arrays

In order to avoid OOM issues

In [ ]:
# Saved the predicted patches as npy in order to avoid OOM issue
save_path = "/content/drive/MyDrive/Python Elective Project/Prediction Phase/Arrays/A1_int.npy"
saved_array = np.save(save_path, prediction)

In [ ]:
# Load the saved file "{}.npy" after saving if the notebook crashes 
# NO NEED TO RUN THIS CELL IF THE NOTEBOOK DOES NOT CRASH AFTER STITCHING THE PREDICTED PATCHES
datum = np.load(save_path)

time: 41.8 s (started: 2021-04-03 15:03:36 +00:00)


In [ ]:
# Check the shape of loaded array
print(f'The shape of the loaded data is {datum.shape}')

The shape of the loaded data is (2891, 512, 512, 1)
time: 1.06 ms (started: 2021-04-03 15:04:27 +00:00)


## Stitch predicted patch images 
Stich the predicted image into one single image that is almost same as the satellite image

In [ ]:
# Save only the number of rows and columns according to the test image
nrows, ncols = new_image[:,:,0].shape
PATCHSIZE = 512
# Iterate loop to generate a combined prediction image from the many predicted image patches 
combo = []
patch_col = math.floor(ncols/PATCHSIZE) 
patch_row = math.floor(nrows/PATCHSIZE)

for i in trange(patch_row):
  patch = np.concatenate(datum[patch_col*i:patch_col*(i+1)], axis=1) 
  combo.append(patch)

join = np.concatenate(combo, axis=0)
stacked_image = join[:,:,0]

100%|██████████| 49/49 [00:00<00:00, 62.59it/s]


time: 3.9 s (started: 2021-04-03 15:06:21 +00:00)


In [ ]:
#CIEW
print(f"Total number of rows and columns for the stitched predicted image: {stacked_image.shape}")

Total number of rows and columns for the stitched predicted image: (25088, 30208)
time: 1.66 ms (started: 2021-04-03 15:06:26 +00:00)


## Window Tranformation
To add geo-reference to the predicted image from the satellite image

In [ ]:
# Size of pixels of the predicted stacked image
xsize, ysize = stacked_image[:,:].shape

# Generate a random window location / Comes from the OG image
xmin, xmax = 0, src.width - xsize
ymin, ymax = 0, src.height - ysize
xoff, yoff = 0, 0  #random.randint(xmin, xmax), random.randint(ymin, ymax)

# Create the window and calculate the transformation objects from the source data (OG image)
window = Window(xoff, yoff, xsize, ysize)
transform = src.window_transform(window)

# Update the profile of the new windowed image
profile = src.profile # Comes from the OG image
src.profile.update({
    "height": ysize,
    "width": xsize,
    "transform": transform
})

time: 6.68 ms (started: 2021-04-03 15:06:32 +00:00)


In [ ]:
# Export the geo-referenced predicted image as a tiff file
new_transform = src.meta["transform"]
new_crs = src.meta["crs"]

new_tiff = rasterio.open("/content/drive/MyDrive/Python Elective Project/Prediction Phase/Images classified/A1_int.tif",
                         mode = "w",
                         height = stacked_image.shape[0],
                         width = stacked_image.shape[1],
                         driver = "GTiff",
                         count = 1,
                         dtype = str(stacked_image.dtype), # Here, the dtype comes from the stacked predicted image
                         crs = new_crs,
                         transform = new_transform)
new_tiff.write(stacked_image, 1)
new_tiff.close()
print("Geo-reference Transformation Successful !!!")

Geo-reference Transformation Successful !!!
time: 24.7 s (started: 2021-04-03 15:06:35 +00:00)


## Compress image size

Use LZW compression to reduce the size of the predicted images

In [ ]:
# Input destination
input_dst = "/content/drive/MyDrive/Kushanav MSc Thesis shared folder/Local Dataset/All tiles/Test images/Entire Study Area/Output predictions/Integer Predictions (Latest-03-04-21)/A1_int.tif"

# Built a VRT and then translate it into a GTIFF using GDAL
gdal.BuildVRT("A1_int.vrt", glob.glob(input_dst))

# Output destination
output_dst = "/content/drive/MyDrive/Kushanav MSc Thesis shared folder/Local Dataset/All tiles/Test images/Entire Study Area/Output predictions/Integer Predictions (Latest-03-04-21)/A1_int_LZW.tif"

time: 166 ms (started: 2021-04-03 15:08:30 +00:00)


In [ ]:
# GDAL translate options
opt = gdal.TranslateOptions(format="GTiff", creationOptions=["TFW=YES", "COMPRESS=LZW"])

# Export as a tiff
if gdal.Translate(output_dst, "A1_int.vrt", options=opt):
    print("Compression Successful !!!")
else:
    print("Compression Failed :(")

Compression Successful !!!
time: 15.5 s (started: 2021-04-03 15:08:32 +00:00)
